# Section of region of interest

Load the file at a low resolution and add regions of interest.

### File selection
Select an image to annotate


In [ ]:
from pathlib import Path
from ipyfilechooser import FileChooser
fc = FileChooser()
display(fc)

Load the file from disk

In [ ]:
from imaris_ims_file_reader.ims import ims
import dask.array as da

path = Path(fc.selected)
folder = path.parent
destdir = Path('./')
#destdir = folder
filename = path.name

resolution_level = 3 

store = ims(folder / filename, ResolutionLevelLock=resolution_level, aszarr=True)

img = da.from_zarr(store)
nuclei = da.max(img[0,3],0).compute().squeeze()

Load the file in napari and add some regions

In [ ]:
import napari
v = napari.view_image(nuclei)

Export the region

In [ ]:
import json
poly = {
    'shape': nuclei.shape,
    'resolution_level': resolution_level,
    'regions': [ p.tolist() for p in v.layers[-1].data ] 
}

with open(destdir/f'{path.stem}-regions.json', "w") as outfile:
    json.dump(poly, outfile)

Reload the regions

In [ ]:

with open(destdir/f'{path.stem}-regions.json', "r") as outfile:
    poly2 = json.load(outfile)

rois = napari.layers.Shapes(poly2['regions'],shape_type='polygon').to_labels(labels_shape=poly2['shape'])


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(nuclei)
plt.imshow(rois,alpha=0.5)
plt.axis('off');